# Dataset Download

In [3]:
import kagglehub
import os

# Get the current working directory (where the notebook is running)
current_dir = os.getcwd()

# Download the latest version to the current working directory
path = kagglehub.dataset_download("jessicali9530/celeba-dataset", path=current_dir)

print("Path to dataset files:", path)

ModuleNotFoundError: No module named 'kagglehub'